In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
sun_data = pd.read_csv("train.csv")
sun_data.drop(['Day', 'Hour','Minute'], axis='columns', inplace=True)
sun_data.head()

,DHI,DNI,WS,RH,T,TARGET
0,0,0,1.5,69.08,-12,0.0
1,0,0,1.5,69.06,-12,0.0
2,0,0,1.6,71.78,-12,0.0
3,0,0,1.6,71.75,-12,0.0
4,0,0,1.6,75.20,-12,0.0


In [ ]:
sun_data.tail()

,DHI,DNI,WS,RH,T,TARGET
52555,0,0,2.4,70.70,-4,0.0
52556,0,0,2.4,66.79,-4,0.0
52557,0,0,2.2,66.78,-4,0.0
52558,0,0,2.1,67.72,-4,0.0
52559,0,0,2.1,67.70,-4,0.0


In [ ]:
print(1092/7)
print(48*7)

156.0
336


In [ ]:
def create_data(data):
    data_X, data_Y = [], []
#     input_data_X, input_data_Y = [], []
    data_X = np.array(sun_data.loc[:52415, ["DHI", "DNI", "WS", "RH", "T"]].values)
    data_Y = np.array(sun_data.loc[:52415, ["TARGET"]].values)
#     for i in range(1092//7):
#         input_data_X += data_X[i*(48*7) : (i+1)*(48*7)]
#         input_data_Y += data_Y[i*(48*7) : (i+1)*(48*7)]
    return data_X, data_Y
#     return input_data_X, input_data_Y

X, Y = create_data(sun_data)
X[:10], Y[:10]

(array([[  0.  ,   0.  ,   1.5 ,  69.08, -12.  ],
        [  0.  ,   0.  ,   1.5 ,  69.06, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  71.78, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  71.75, -12.  ],
        [  0.  ,   0.  ,   1.6 ,  75.2 , -12.  ],
        [  0.  ,   0.  ,   1.5 ,  69.29, -11.  ],
        [  0.  ,   0.  ,   1.5 ,  72.56, -11.  ],
        [  0.  ,   0.  ,   1.4 ,  72.55, -11.  ],
        [  0.  ,   0.  ,   1.3 ,  74.62, -11.  ],
        [  0.  ,   0.  ,   1.3 ,  74.61, -11.  ]]), array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

In [ ]:
print(len(X), len(Y))
print(len(X[0]), len(Y[0]))
X.shape, Y.shape

52416 52416
5 1


((52416, 5), (52416, 1))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
input_scaler = scaler.fit(X)
output_scaler = scaler.fit(Y)

X = input_scaler.transform(X)
Y = output_scaler.transform(Y)

# X = X.reshape(-1,336,5)
# Y = Y.reshape(-1,7,1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
X_train.shape, y_train.shape

((39312, 5), (39312, 1))

In [ ]:
def create_dataset (X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
X_train, y_train = create_dataset(X_train, y_train, 24)
X_test, y_test= create_dataset(X_test, y_test, 24)

In [ ]:
print('X_train.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_train.shape)

X_train.shape:  (13080, 24, 5)
y_train.shape:  (39288, 1)
X_test.shape:  (13080, 24, 5)
y_test.shape:  (39288, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, Bidirectional, GRU
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(GRU(64, dropout=0.3, return_sequences=True, input_shape=[24,5]))
model.add(GRU(64, dropout=0.3))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="mean_squared_error", optimizer = "adam", metrics=["accuracy"])

filepath = "weights/" + "lstm_model" + "-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(X_train, y_train, epochs=200, batch_size=128, callbacks=callbacks_list, validation_split=0.1)

Epoch 1/200
277/277 [==============================] - 16s 47ms/step - loss: 1.0229 - accuracy: 0.0000e+00 - val_loss: 0.9927 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to weights/lstm_model-01-0.0000.hdf5
Epoch 2/200
277/277 [==============================] - 12s 44ms/step - loss: 0.9928 - accuracy: 0.0000e+00 - val_loss: 0.9927 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/200
131/277 [=============>................] - ETA: 6s - loss: 0.9852 - accuracy: 0.0000e+00

In [ ]:
def plot_loss (history):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
plot_loss (history)
